# Step 1: Setup

In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.12.0'

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Step 2: Importing the dataset from kaggle


In [4]:
# install kaggle API
! pip install -q kaggle

In [5]:
# create a directory
! mkdir -p ~/.kaggle

In [6]:
# import kaggle api
from google.colab import files
upload = files.upload()

Saving kaggle.json to kaggle.json


In [7]:
# copy API key to kaggle dir
! cp kaggle.json ~/.kaggle

In [8]:
# disable api key
! chmod  600 ~/.kaggle/kaggle.json

In [9]:
# import the dataset
! kaggle datasets download -d tongpython/cat-and-dog

 99% 216M/218M [00:01<00:00, 132MB/s]
100% 218M/218M [00:01<00:00, 148MB/s]


In [10]:
# unzipping the dataser
! unzip -q /content/cat-and-dog.zip

In [11]:
train_dir = '/content/training_set/training_set'
test_dir = '/content/test_set/test_set'

# Step 3: Building the model

In [12]:
# reshape the images
img_shape = (128,128,3)

## Loading the Pre-Trained model (MobileNetV2)

In [13]:
base_model = tf.keras.applications.MobileNetV2(input_shape = img_shape, include_top=False, weights='imagenet')

9406464/9406464 [==============================] - 0s 0us/step


In [14]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][

In [15]:
# freezing the model
base_model.trainable = False

## Defining the custom head for the neetwork

In [16]:
base_model.output

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

In [17]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [18]:
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [19]:
# Output layer/ prediction layer
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

In [20]:
prediction_layer

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dense')>

## Define the transfer learning model

In [21]:
model = tf.keras.models.Model(inputs = base_model.input, outputs=prediction_layer)

In [23]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][0]']           

In [24]:
# compile the model
opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001)

In [25]:
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

## Create Data Generators

In [27]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [28]:
data_gen_train = ImageDataGenerator(rescale=1/255.0)
data_gen_test = ImageDataGenerator(rescale=1/255.0)

In [30]:
train_generator = data_gen_train.flow_from_directory(directory=train_dir, target_size=(128,128), batch_size=128, class_mode='binary')

Found 8005 images belonging to 2 classes.


In [31]:
test_generator = data_gen_test.flow_from_directory(directory=test_dir, target_size=(128,128), batch_size=128, class_mode='binary')

Found 2023 images belonging to 2 classes.


# Step 4: Training the model

In [32]:
model.fit_generator(generator=train_generator, epochs=5, validation_data=test_generator)

<ipython-input-32-18b18ca5977c>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator, epochs=5, validation_data=test_generator)


Epoch 1/5
63/63 [==============================] - 183s 3s/step - loss: 0.7172 - accuracy: 0.5884 - val_loss: 0.5270 - val_accuracy: 0.7449
Epoch 2/5
63/63 [==============================] - 165s 3s/step - loss: 0.4222 - accuracy: 0.8245 - val_loss: 0.3300 - val_accuracy: 0.8853
Epoch 3/5
63/63 [==============================] - 170s 3s/step - loss: 0.2784 - accuracy: 0.9042 - val_loss: 0.2295 - val_accuracy: 0.9209
Epoch 4/5
63/63 [==============================] - 174s 3s/step - loss: 0.2064 - accuracy: 0.9293 - val_loss: 0.1791 - val_accuracy: 0.9397
Epoch 5/5
63/63 [==============================] - 171s 3s/step - loss: 0.1691 - accuracy: 0.9409 - val_loss: 0.1509 - val_accuracy: 0.9471


# Step 5: Fine Tuning

In [33]:
base_model.trainable= True

In [35]:
len(base_model.layers)

154

In [36]:
fine_tune_at = 100

In [38]:
# freeze the layers before 100
for layer in base_model.layers[:fine_tune_at]:
  base_model.trainable= False

In [39]:
# Compile the model again
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [40]:
model.fit_generator(generator=train_generator, epochs=5, validation_data=test_generator)

<ipython-input-40-18b18ca5977c>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator, epochs=5, validation_data=test_generator)


Epoch 1/5
63/63 [==============================] - 184s 3s/step - loss: 0.1472 - accuracy: 0.9475 - val_loss: 0.1350 - val_accuracy: 0.9550
Epoch 2/5
63/63 [==============================] - 172s 3s/step - loss: 0.1332 - accuracy: 0.9528 - val_loss: 0.1218 - val_accuracy: 0.9585
Epoch 3/5
63/63 [==============================] - 167s 3s/step - loss: 0.1238 - accuracy: 0.9553 - val_loss: 0.1143 - val_accuracy: 0.9614
Epoch 4/5
63/63 [==============================] - 190s 3s/step - loss: 0.1165 - accuracy: 0.9575 - val_loss: 0.1081 - val_accuracy: 0.9624
Epoch 5/5
63/63 [==============================] - 184s 3s/step - loss: 0.1112 - accuracy: 0.9590 - val_loss: 0.1050 - val_accuracy: 0.9634
